In [ ]:
catalog = pystac_client.Client.open(
    "https://planetarycomputer.microsoft.com/api/stac/v1",
    modifier=planetary_computer.sign_inplace,
)

search = catalog.search(collections=["io-lulc-9-class"], bbox=mendocino_bbox)

items = search.item_collection()
print(f"Returned {len(items)} Items")

In [ ]:
import stackstac

# The STAC metadata contains some information we'll want to use when creating
# our merged dataset. Get the EPSG code of the first item and the nodata value.
item = items[0]

# Create a single DataArray from out multiple resutls with the corresponding
# rasters projected to a single CRS. Note that we set the dtype to ubyte, which
# matches our data, since stackstac will use float64 by default.
stack = (
    stackstac.stack(
        items,
        dtype=float,
        fill_value=255,
        bounds_latlon=mendocino_bbox,
        sortby_date=False,
        epsg=4326
    )
    .assign_coords(
        time=pd.to_datetime([item.properties["start_datetime"] for item in items])
        .tz_convert(None)
        .to_numpy()
    )
    .sortby("time")
)

stack

In [ ]:
import dask.distributed

client = dask.distributed.Client(processes=False)
print(f"/proxy/{client.scheduler_info()['services']['dashboard']}/status")

In [ ]:
# Filter the stack by year before computing
stack_filtered = stack.sel(time=stack.time.dt.year.isin([2017, 2019]))

print(f"Original stack shape: {stack.shape}")
print(f"Filtered stack shape: {stack_filtered.shape}")

# Now compute only the filtered data
merged = stack_filtered.squeeze().compute()

print(f"Final merged data shape: {merged.shape}")
print(f"Years in merged data: {merged.time.dt.year.values}")

In [ ]:
# g = merged.plot(col="time")
# for ax in g.axs.flat:
#     ax.set_axis_off()


In [ ]:
from pystac.extensions.item_assets import ItemAssetsExtension

collection = catalog.get_collection("io-lulc-9-class")
ia = ItemAssetsExtension.ext(collection)

x = ia.item_assets["data"]
class_names = {x["summary"]: x["values"][0] for x in x.properties["file:values"]}
values_to_classes = {v: k for k, v in class_names.items()}
class_count = len(class_names)
class_names

In [ ]:
with rasterio.open(item.assets["data"].href) as src:
    colormap_def = src.colormap(1)  # get metadata colormap for band 1
    colormap = [
        np.array(colormap_def[i]) / 255 for i in range(max(class_names.values()) + 1)
    ]  # transform to matplotlib color format

cmap = ListedColormap(colormap)

In [ ]:
vmin = 0
vmax = max(class_names.values()) + 1
epsg = merged.epsg.item()
# Convert to a standard projection before plotting
merged = merged.rio.reproject("EPSG:4326")

p = merged.plot(
    subplot_kws=dict(projection=ccrs.PlateCarree()),
    col="time",
    transform=ccrs.PlateCarree(),
    cmap=cmap,
    vmin=vmin,
    vmax=vmax,
    figsize=(16, 6),
)
ticks = np.linspace(0.5, vmax - 0.5, vmax - vmin)
labels = [values_to_classes.get(i, "") for i in range(cmap.N)]
p.cbar.set_ticks(ticks, labels=labels)
p.cbar.set_label("Class")

In [ ]:
colors = list(cmap.colors)

ax = (
    pd.value_counts(merged.data.ravel(), sort=False)
    .sort_index()
    .reindex(range(cmap.N), fill_value=0)
    .rename(values_to_classes)
    .plot.barh(color=colors, rot=0, width=0.9)
)
ax.set(
    title="Distribution of Land Cover classes",
    ylabel="Landcover class",
    xlabel="Class count",
)